# Optimización de Portafolio

In [ ]:
import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical operations
import os  # For interacting with the operating system (e.g., file paths)
import yfinance as yf  # For fetching financial data from Yahoo Finance

from pypfopt.efficient_frontier import EfficientFrontier  # For creating efficient frontier and optimizing portfolios
from pypfopt import risk_models, expected_returns  # For calculating risk models (e.g., covariance matrix)
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices  # For discrete allocation of portfolio weights

from bs4 import BeautifulSoup  # For parsing HTML and XML documents
from datetime import datetime  # For manipulating dates and times

### Create csv

In [2]:
def read_stock_symbols(file_path):
    with open(file_path, 'r') as file:
        return file.read().splitlines()

In [3]:
def fetch_adj_close_prices(symbols):
    adj_close_list = []
    for symbol in symbols:
        stock = yf.Ticker(symbol)
        hist = stock.history(period="max")
        
        # Reset the index to work with 'Date'
        hist.reset_index(inplace=True)
        
        # Convert 'Date' column to datetime and set it to UTC to standardize time zones
        hist['Date'] = pd.to_datetime(hist['Date']).dt.tz_localize(None)  # Remove time zone info
        
        # Group by date to remove time component and get the last close price of the day
        daily_data = hist.groupby(hist['Date'].dt.date).agg({'Close': 'last'}).rename(columns={'Close': symbol})
        
        adj_close_list.append(daily_data)
    
    return pd.concat(adj_close_list, axis=1)

In [4]:
def generate_all_dates(start_date, end_date):
    # Generate a date range with Daily frequency
    return pd.date_range(start=start_date, end=end_date, freq='D')

In [ ]:
current_date = datetime.now().strftime("%Y-%m-%d")
csv_file = f"{current_date}.csv"

if not os.path.exists(csv_file):
    stock_symbols = read_stock_symbols('ETFS&Stocks.txt')
    # stock_symbols.insert(0, '^GSPC')  # Add S&P 500 symbol
    
    df = fetch_adj_close_prices(stock_symbols)

    min_date = df.index.min()
    max_date = df.index.max()

    all_dates = pd.DataFrame(generate_all_dates(min_date, max_date), columns=['Date'])

    # Convert 'Date' column to datetime to match the main dataframe
    all_dates['Date'] = pd.to_datetime(all_dates['Date'])

    # Merge the stock data with the full range of dates (business days)
    df.reset_index(inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])  # Ensure 'Date' column is in datetime format

    merged_df = all_dates.merge(df, on='Date', how='left')
    merged_df.set_index('Date', inplace=True)

    # Save to CSV
    merged_df.to_csv(csv_file, index=True)

# Read the resulting CSV file
df = pd.read_csv(csv_file, index_col='Date', parse_dates=True)

C:\Users\herie\AppData\Local\Temp\ipykernel_19464\1594069110.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.reset_index(inplace=True)


In [6]:
print(df.columns[df.isnull().all()])

Index([], dtype='object')


In [7]:
portfolio = 100000
days = 9
rf = 0

In [8]:
rf = (1 + rf) ** (252/days) - 1

In [9]:
from openpyxl import load_workbook

def save_to_excel(file_path, expected_return, volatility, sharpe_ratio, rf, leftover, type):
    
    workbook = load_workbook(file_path)
    sheet = workbook.active

    sheet["E1"] = type
    sheet["E2"] = "Days"
    sheet["E3"] = "Portafolio value"
    sheet["E4"] = "Return"
    sheet["E5"] = "Volatility"
    sheet["E6"] = "Sharpe Ratio"
    sheet["E7"] = "Risk-Free Rate"
    sheet["E8"] = "Leftover"
    sheet["F2"] = days
    sheet["F3"] = portfolio
    sheet["F4"] = expected_return
    sheet["F5"] = volatility
    sheet["F6"] = sharpe_ratio
    sheet["F7"] = rf
    sheet["F8"] = leftover

    workbook.save(file_path)

In [10]:
mu = expected_returns.mean_historical_return(df, frequency=days)
s = risk_models.sample_cov(df, frequency=days)

c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\pypfopt\expected_returns.py:56: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna(how="all")
c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\pypfopt\expected_returns.py:56: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna(how="all")
c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\pypfopt\risk_models.py:70: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.

The EfficientFrontier class is used to create an efficient frontier, which is a set of optimal portfolios that offer the highest expected return for a defined level of risk or the lowest risk for a given level of expected return.

In [11]:
ef = EfficientFrontier(mu, s)

We will use this later for the Excel.

In [ ]:
type = "Max Return"

This line optimizes the portfolio to maximize the Sharpe ratio.

In [ ]:
weights = ef.max_quadratic_utility()

# weights = ef.max_sharpe(risk_free_rate=rf)
# weights = ef.efficient_risk(target_volatility)
# weights = ef.efficient_return(target_return)
# weights = ef.min_volatility()
# weights = ef.max_quadratic_utility()

This line cleans the optimized weights by rounding them and setting small weights to zero.
The result is a more interpretable and practical set of weights.

In [ ]:
cleaned_weights = ef.clean_weights()

This line calculates the expected return, volatility, and Sharpe ratio of the optimized portfolio and assigns these values to the variables expected_return, volatility, and sharpe_ratio, respectively. The verbose=True parameter ensures that these metrics are also printed to the console.

In [ ]:
expected_return, volatility, sharpe_ratio = ef.portfolio_performance(verbose=True, risk_free_rate=rf)

This code snippet retrieves the latest prices of the assets, initializes a DiscreteAllocation object with the cleaned weights and total portfolio value, and then performs the discrete allocation to determine the number of shares to buy for each asset and the leftover cash.

In [ ]:
latest_prices = get_latest_prices(df)
da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portfolio)
allocation, leftover = da.lp_portfolio()

This code snippet creates a list of the number of shares allocated to each company and then constructs a pandas DataFrame with two columns: 'Company Ticker' and 'Discrete Allocation'. The DataFrame provides a clear and organized view of the portfolio's allocation.

In [ ]:
# Get company names for the allocation
discrete_allocation_list = [allocation[symbol] for symbol in allocation]

# Create a DataFrame for the portfolio
portfolio_df = pd.DataFrame({
    'Company Ticker': allocation.keys(),
    'Discrete Allocation': discrete_allocation_list
})

Create the Excel.

In [ ]:
current_date = datetime.now().strftime("%d-%m-%Y-%H-%M")

file_path = f"Portafolios/{type} {current_date}.xlsx"
portfolio_df.to_excel(file_path, index=False)

save_to_excel(file_path, expected_return, volatility, sharpe_ratio, rf, leftover, type)